## Hosptial Mortality Classifcation
this notebookes creates classifers that predict probablity that a patient died in the hospital based on lab values. It uses Phyisio Mimic III as a data source and uses the python evalML to evaluate classifers. m

In [28]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import numpy as np
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.model_selection import cross_validate,  StratifiedKFold
from sklearn.metrics import *

from evalml.automl import AutoMLSearch
import evalml
import os
import re
import mlflow
from evalml.model_understanding.prediction_explanations import explain_predictions
from mlflow.models.signature import infer_signature
import json
import os
import warnings
from prince import MCA

MAX_MEMORY = "32g"
data_dir = os.getenv('PHYSIO_HOME')

#### Data Loading
Data is loaded from Phyiso MimiIII amd saved as paquet to pyspark data frames

#### Data Egneineering 
creates a features data frame using max and min lab values during hospital stays

In [29]:
# reads all the csvs and writes them to parquet filesspark = SparkSession.builder \
spark = SparkSession.builder \
    .appName("HostpitalMortalityClassifier") \
    .config("spark.executor.memory", MAX_MEMORY) \
    .config("spark.driver.memory", MAX_MEMORY) \
    .getOrCreate()

LABEVENTS =  spark.read.parquet(data_dir + '/LABEVENTS.parquet')
D_LABITEMS =  spark.read.parquet(data_dir + '/D_LABITEMS.parquet')
ADMISSIONS =   spark.read.parquet(data_dir + '/ADMISSIONS.parquet')


# sets the number of features to section by frequency
n_features = 100

# gets the top n_features most frequent features 
top_features = LABEVENTS\
                .join(D_LABITEMS, on = 'ITEMID', how='inner')\
                .dropna(subset=['VALUENUM'])\
                .groupby('LABEL')\
                .count().sort('count', ascending=False)\
                .limit(n_features).drop('count')


## Data Transformations 
## gets the max and min value from the top n_features
## groups by hospital admit id
## creates a flag where the patient died "Expired" in the hosptial                                        
data = LABEVENTS\
.join(D_LABITEMS, on = 'ITEMID', how='inner')\
.join(top_features, on='label', how='inner')\
.dropna(subset=['VALUENUM'])\
.groupby('HADM_ID')\
.pivot('LABEL')\
.agg(max('VALUENUM').alias('max'), min('VALUENUM').alias('min'))\
.join(ADMISSIONS.select('HADM_ID', col('HOSPITAL_EXPIRE_FLAG').alias('label')), on='HADM_ID', how='inner')\
.filter('label in (0,1)')

## data Extraction to Pandas
df = data.toPandas().set_index('HADM_ID')

## create arrays for training model 
y = df.loc[:, 'label'].values
X = df.drop('label', axis=1).values

n_rows = X.shape[0]
n_features = X.shape[1]
feature_names_all = np.array(list(df.drop('label', axis=1).columns))
label_prob = y.mean()
print(F' n_rows: {n_rows}, n_features: {n_features}, label_prob {np.round(label_prob , 3)}')
print(F'features: {feature_names_all}')

spark.stop()

 n_rows: 58144, n_features: 200, label_prob 0.1
features: ['% Hemoglobin A1c_max' '% Hemoglobin A1c_min'
 'Alanine Aminotransferase (ALT)_max' 'Alanine Aminotransferase (ALT)_min'
 'Albumin_max' 'Albumin_min' 'Alkaline Phosphatase_max'
 'Alkaline Phosphatase_min' 'Alveolar-arterial Gradient_max'
 'Alveolar-arterial Gradient_min' 'Amylase_max' 'Amylase_min'
 'Anion Gap_max' 'Anion Gap_min' 'Asparate Aminotransferase (AST)_max'
 'Asparate Aminotransferase (AST)_min' 'Atypical Lymphocytes_max'
 'Atypical Lymphocytes_min' 'Bands_max' 'Bands_min' 'Base Excess_max'
 'Base Excess_min' 'Basophils_max' 'Basophils_min' 'Bicarbonate_max'
 'Bicarbonate_min' 'Bilirubin, Direct_max' 'Bilirubin, Direct_min'
 'Bilirubin, Indirect_max' 'Bilirubin, Indirect_min'
 'Bilirubin, Total_max' 'Bilirubin, Total_min' 'CK-MB Index_max'
 'CK-MB Index_min' 'Calcium, Total_max' 'Calcium, Total_min'
 'Calculated Total CO2_max' 'Calculated Total CO2_min' 'Chloride_max'
 'Chloride_min' 'Chloride, Whole Blood_max' 'Chlo

#### Basic Data Statics


In [30]:
data_stats_path = 'data_stats.csv'
data_stats = df.describe()
data_stats.to_csv(data_stats_path)
data_stats 

,% Hemoglobin A1c_max,% Hemoglobin A1c_min,Alanine Aminotransferase (ALT)_max,Alanine Aminotransferase (ALT)_min,Albumin_max,Albumin_min,Alkaline Phosphatase_max,Alkaline Phosphatase_min,Alveolar-arterial Gradient_max,Alveolar-arterial Gradient_min,...,White Blood Cells_min,pCO2_max,pCO2_min,pH_max,pH_min,pO2_max,pO2_min,tacroFK_max,tacroFK_min,label
count,6614.000000,6614.000000,33661.000000,33661.000000,30996.000000,30996.000000,33573.000000,33573.000000,9884.000000,9884.000000,...,56990.000000,37347.000000,37347.000000,47548.000000,47548.000000,37350.000000,37350.000000,921.000000,921.000000,58144.000000
mean,6.630215,6.614914,156.416149,42.653828,3.373439,3.013466,150.903524,101.232240,475.819102,412.899332,...,8.319279,51.336493,35.295204,7.264455,5.964260,256.436573,85.320214,14.397611,4.427904,0.100182
std,1.778103,1.775684,704.995144,163.113849,0.662730,0.747922,187.566340,98.089175,135.059334,132.920845,...,6.470020,16.686125,9.421851,0.709379,1.015657,148.659425,63.776929,7.338076,2.601365,0.300246
min,3.800000,3.800000,0.000000,0.000000,1.000000,0.900000,3.000000,0.000000,6.000000,-22.000000,...,0.000000,8.000000,0.000000,1.000000,0.000000,0.000000,0.000000,1.200000,1.200000,0.000000
25%,5.600000,5.600000,18.000000,13.000000,2.900000,2.500000,69.000000,57.000000,380.000000,310.000000,...,5.400000,42.000000,30.000000,7.350000,5.000000,124.000000,46.000000,9.500000,2.700000,0.000000
50%,6.000000,6.000000,30.000000,21.000000,3.400000,3.000000,96.000000,76.000000,510.000000,421.000000,...,7.200000,48.000000,34.000000,7.440000,5.500000,230.000000,70.000000,14.100000,3.800000,0.000000
75%,6.900000,6.900000,66.000000,35.000000,3.800000,3.600000,156.000000,108.000000,590.000000,524.000000,...,9.700000,56.000000,39.000000,7.500000,7.000000,390.000000,100.000000,18.300000,5.500000,0.000000
max,22.000000,22.000000,25460.000000,7035.000000,6.900000,5.700000,4695.000000,3658.000000,794.000000,726.000000,...,378.000000,247.000000,165.000000,10.000000,9.000000,1914.000000,630.000000,93.200000,25.700000,1.000000


In [31]:
from sklearn.feature_selection import RFE
from sklearn.tree import DecisionTreeClassifier
from FeatureSelectors.feature_extraction import IsObserved
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from FeatureSelectors.feature_extraction import Scaler

#### Data Splitting
Data splitting via Statified Shuffle Split

#### Feature Selection pyImpetous 

In [32]:
# data splitting
n_mca_comps = 10

splitter = StratifiedKFold(shuffle=True)
train_index, test_index = next(splitter.split(X, y))

## tran
obs_pipe = Pipeline(steps=[('Isob', IsObserved()), ('mca', MCA(n_mca_comps))])\
.fit(df.iloc[train_index, :].loc[:, feature_names_all])

num_pipe = Pipeline(steps=[('scaler', Scaler()), ('imp', SimpleImputer())])\
.fit(df.iloc[train_index, :].loc[:, feature_names_all].values)


indicies_all = np.arange(len(feature_names_all))

transformers = [('obs',obs_pipe, indicies_all), ('imp', num_pipe, indicies_all)]   


pipe = ColumnTransformer(transformers).fit(df.iloc[train_index, :].loc[:, feature_names_all].values )

X_train = pipe.transform(df.iloc[train_index, :].loc[:, feature_names_all] )
X_test =  pipe.transform(df.iloc[test_index, :].loc[:, feature_names_all] )
y_train = y[train_index]
y_test = y[test_index]

feature_names_transformed_all = np.array(['mca_'+ str(i) for i in range(n_mca_comps)] + list(feature_names_all))

In [33]:
pipeline_desc = F'concat[{pipe.transformers[0][1].steps},  {pipe.transformers[1][1].steps} ]' 
pipeline_desc

"concat[[('Isob', IsObserved()), ('mca', MCA(n_components=10))],  [('scaler', Scaler()), ('imp', SimpleImputer())] ]"

In [34]:
from FeatureSelectors.Distancecorrelation import Selector
selector = Selector()
selector= selector.fit(X_train, y_train)
support_index = selector.get_support()
best_features = feature_names_transformed_all[support_index]

In [ ]:

best_features

#### Modeliing Fitting Using AutoML
Searchs through models to find best 

In [36]:

automl = AutoMLSearch(X_train=pd.DataFrame(X_train,columns=feature_names_transformed_all).loc[:, best_features], y_train=y_train, problem_type='binary')
with  warnings.catch_warnings():
    warnings.filterwarnings('ignore')
    automl.search()
model = automl.best_pipeline


Using default limit of max_batches=1.

Generating pipelines to search over...
8 pipelines ready for search.

*****************************
* Beginning pipeline search *
*****************************

Optimizing for Log Loss Binary. 
Lower score is better.

Using SequentialEngine to train and score pipelines.
Searching up to 1 batches for a total of 9 pipelines. 
Allowed model families: decision_tree, xgboost, catboost, random_forest, extra_trees, lightgbm, linear_model



FigureWidget({
    'data': [{'mode': 'lines+markers',
              'name': 'Best Score',
              'type'…

Evaluating Baseline Pipeline: Mode Baseline Binary Classification Pipeline
Mode Baseline Binary Classification Pipeline:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 3.460

*****************************
* Evaluating Batch Number 1 *
*****************************

Elastic Net Classifier w/ Imputer + Undersampler + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.273
Decision Tree Classifier w/ Imputer + Undersampler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.309
Random Forest Classifier w/ Imputer + Undersampler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.248
LightGBM Classifier w/ Imputer + Undersampler:
	Starting cross validation
	Finished cross validation - mean Log Loss Binary: 0.186
Logistic Regression Classifier w/ Imputer + Undersampler + Standard Scaler:
	Starting cross validation
	Finished cross validation - mean Log Loss Bi

#### Model Performance
Calcuates Model Peformace on Test Set

In [37]:
# predicts the test data
test_preds = model.predict_proba(pd.DataFrame(X_test,columns=feature_names_transformed_all).loc[:, best_features]).iloc[:, 1]
test_pred_labels = model.predict(pd.DataFrame(X_test,columns=feature_names_transformed_all).loc[:, best_features])

## predicts the training data 
train_preds = model.predict_proba(pd.DataFrame(X_train,columns=feature_names_transformed_all).loc[:, best_features]).iloc[:, 1]
train_pred_labels = model.predict(pd.DataFrame(X_train,columns=feature_names_transformed_all).loc[:, best_features])



# calcuates metrics on test data
test_f1 = f1_score(y_test, test_pred_labels)
test_acc_balanced = balanced_accuracy_score(y_test, test_pred_labels)
test_acc = accuracy_score(y_test, test_pred_labels)
test_precision = precision_score(y_test, test_pred_labels)
test_recall = recall_score(y_test, test_pred_labels)
test_auc_score = roc_auc_score(y[test_index], test_preds)
print(F'roc_auc_score: {test_auc_score } on test')


# calculates metrics on training data 
train_f1 = f1_score(y_train, train_pred_labels)
train_acc_balanced = balanced_accuracy_score(y_train, train_pred_labels)
train_acc = accuracy_score(y_train, train_pred_labels)
train_precision = precision_score(y_train, train_pred_labels)
train_recall = recall_score(y_train, train_pred_labels)
train_auc_score = roc_auc_score(y_train, train_preds)
print(F'roc_auc_score: {train_auc_score} on train')

# gets params Artifacts for logging mlflow model
n_cases = np.sum(y == 1)
n_controls = np.sum(y == 0)
n_train_obs = X_train.shape[0]
n_test_obs = X_test.shape[0]

train_label_prob = y_train.mean()
test_label_prob = y_test.mean()
desc = str(model.describe())
model_type = type(model)
split_type = type(splitter)
input_example = pd.DataFrame(X_train,columns=feature_names_transformed_all).loc[:, best_features].head(5).fillna(0)
signature = infer_signature(input_example,  model.predict_proba(input_example))
n_features =input_example.shape[1]

roc_auc_score: 0.9399788032707275 on test
roc_auc_score: 0.9753676882636406 on train

*************************************************
* LightGBM Classifier w/ Imputer + Undersampler *
*************************************************

Problem Type: binary
Model Family: LightGBM
Number of features: 133

Pipeline Steps
1. Imputer
	 * categorical_impute_strategy : most_frequent
	 * numeric_impute_strategy : mean
	 * categorical_fill_value : None
	 * numeric_fill_value : None
2. Undersampler
	 * sampling_ratio : 0.25
	 * min_samples : 100
	 * min_percentage : 0.1
	 * sampling_ratio_dict : None
3. LightGBM Classifier
	 * boosting_type : gbdt
	 * learning_rate : 0.1
	 * n_estimators : 100
	 * max_depth : 0
	 * num_leaves : 31
	 * min_child_samples : 20
	 * n_jobs : -1
	 * bagging_freq : 0
	 * bagging_fraction : 0.9


#### Feature Importance
save feature importance to a dictionary for later logging 

In [38]:
imp = model.feature_importance.set_index('feature')

# dumps feature importance to a dictionary for logging as an artifact
imp_dict = imp.to_dict()['importance']
imp_json_path = 'feature_importance.json'
with open(imp_json_path, 'w') as f:
    json.dump(imp_dict,f)

imp.head()


,importance
feature,
Bicarbonate_max,147
Glucose_min,128
Urea Nitrogen_min,112
pCO2_max,105
Sodium_max,103


#### Model Tracking
Uses an mlflow tracking server to save the model, parameters and metrics

In [44]:
artifact_path = 'Model'
data_grain = 'HADM_ID'
label_name = 'HOSPITAL_EXPIRE_FLAG'
data_source = 'PhysioMimicIII'
run_name = 'evalML_dcor w_isObeserved'
tracking_uri = "http://localhost:5000"
mlflow.set_tracking_uri(tracking_uri)
experiment_id=1
with mlflow.start_run(run_name=run_name) as run:
    
    tracking_uri = mlflow.get_tracking_uri()
    artifact_uri = mlflow.get_artifact_uri()
    print("Tracking uri: {}".format(tracking_uri))
    print("Artifact uri: {}".format(artifact_uri))
    mlflow.sklearn.log_model(model,
                         artifact_path=artifact_path, 
                         signature=signature,
                         input_example=input_example
                        )
    mlflow.log_artifact(imp_json_path)
    mlflow.log_artifact(data_stats_path)
    mlflow.log_param('data_source', data_source)
    mlflow.log_param('label_name', label_name)
    mlflow.log_param('data_grain', data_grain)
    mlflow.log_param('n_cases', n_cases)
    mlflow.log_param('n_controls', n_controls)
    mlflow.log_param('n_train_obs', n_train_obs)
    mlflow.log_param('n_test_obs', n_test_obs)
    mlflow.log_param('n_features', n_features)
    mlflow.log_param('train_label_prob', train_label_prob)
    mlflow.log_param('test_label_prob', test_label_prob)
    mlflow.log_param('desc', desc)
    mlflow.log_param('model_type',model_type)
    mlflow.log_param('split_type',split_type)
    mlflow.log_param('feature_selection', type(selector))
    mlflow.log_param('pipeline_desc', pipeline_desc)
    mlflow.log_param('pipeline_desc', pipeline_desc)
    mlflow.log_param('importance', str(imp))
    mlflow.log_metric('train_f1', train_f1)
    mlflow.log_metric('train_acc_balanced', train_acc_balanced)
    mlflow.log_metric('train_acc', train_acc)
    mlflow.log_metric('train_precision', train_precision)
    mlflow.log_metric('train_recall', train_recall)
    mlflow.log_metric('train_auc_score', train_auc_score)
    mlflow.log_metric('test_f1', test_f1)
    mlflow.log_metric('test_acc_balanced', test_acc_balanced)
    mlflow.log_metric('test_acc', test_acc)
    mlflow.log_metric('test_precision', test_precision)
    mlflow.log_metric('test_recall', test_recall)
    mlflow.log_metric('test_auc_score', test_auc_score)
    mlflow.log_param('features', '|'.join(imp.index))
    run_id = run.info.run_id
    experiment_id = run.info.experiment_id 
    mlflow.end_run()
    print(F'logging experiment_id: "{experiment_id}" run_id :"{run_id}" completed')


Tracking uri: http://localhost:5000
Artifact uri: ./mlruns/0/0f377876f9344b48994db7873e1d3d8b/artifacts
logging experiment_id: "0" run_id :"0f377876f9344b48994db7873e1d3d8b" completed


In [43]:
str(imp)

'                                  importance\nfeature                                     \nBicarbonate_max                          147\nGlucose_min                              128\nUrea Nitrogen_min                        112\npCO2_max                                 105\nSodium_max                               103\n...                                      ...\nProtein, Total_min                         2\nCholesterol, LDL, Calculated_max           1\nEpithelial Cells_min                       1\nFerritin_min                               1\nUrobilinogen_min                           1\n\n[133 rows x 1 columns]'